In [1]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
import visualise_spacy_tree
#from IPython.display import image,display
import PyPDF2

from numpy import loadtxt
import numpy as np
import pandas as pd
import os
import fnmatch
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag,RegexpParser
import nltk

In [2]:
import pdfplumber
import regex as re

In [3]:
def process_segmentation():
    ch=""
    names=""
    start=0
    process_df=pd.DataFrame(columns=['Process Name','start','Corpus'])
    with pdfplumber.open("PMBOK 5th.pdf") as pdf:
        for i in range (374,380):
            page=pdf.pages[i]
            pages=page.extract_text()
            for line in pages.split('\n'):
                if re.match('\d{2}.\d\s[A-Za-z]+\s[A-Za-z]',line):
                    if start!=0:
                        process_df=process_df.append({'Process Name':name,'start':start,'Corpus':ch[1:]},ignore_index=True)
                    ch=''
                    start=i
                    name=line[5:]
                ch=ch+'_ '+line
        process_df=process_df.append({'Process Name':name,'start':start,'Corpus':ch[1:]},ignore_index=True)
        return(process_df)

In [4]:
A=process_segmentation()

In [5]:
A

,Process Name,start,Corpus
0,control risks,374,11.6 control risks_ Control Risks is the proc...


In [8]:
A["lower"] = A['Corpus'].str.lower().str.replace('[^\w\s]','')
new_df = A.lower.str.split(expand=True).stack().value_counts().reset_index()
new_df.columns = ['Word', 'Frequency'] 
new_df.style

,Word,Frequency
0,the,99
1,_,71
2,project,64
3,risk,64
4,of,51
5,and,49
6,management,37
7,to,34
8,risks,33
9,performance,26


In [67]:
ch=""
with pdfplumber.open("PMBOK 5th.pdf")as temp:
    for i in range(374,380):
        first_page=temp.pages[i]
        ch=ch+first_page.extract_text()

In [68]:
ch

'11 - PROJECT RISK MANAGEMENT\n11.6 control risks\nControl Risks is the process of implementing risk response plans, tracking identified risks, monitoring residual \nrisks, identifying new risks, and evaluating risk process effectiveness throughout the project. The key benefit of \nthis process is that it improves efficiency of the risk approach throughout the project life cycle to continuously \noptimize risk responses. The inputs, tools and techniques, and outputs of this process are depicted in Figure 11-20. \nFigure 11-21 depicts the data flow diagram of the process.\nInputs Tools & Techniques Outputs\n.1 Project management plan .1 Risk reassessment .1 Work performance  \n.2 Risk register .2 Risk audits   information\n.3 Work performance data .3 Variance and trend   .2 Change requests\n.4 Work performance     analysis .3 Project management plan  \n  reports .4 Technical performance     updates\n  measurement .4 Project documents  \n.5 Reserve analysis   updates\n.6 Meetings .5 Orga

In [69]:
inputs_titles=re.compile(r'\d+\..+: Inputs\n')
tools_and_techniques=re.compile(r'\d+\..+: tools and techniques\n')
outputs_titles=re.compile(r'\d+\..+: outputs\n')
get_subtitles=re.compile(r'[^|\n](\d+\.){2,4}\d+ [a-z|\s|A-Z]+(\d+)?\n')
title=re.compile(r'\d+ (.+): Inputs\n')

In [70]:
sections = inputs_titles.finditer(ch)
df = []
inputs = []
tt = []
outputss=[]
lst_sec = [elem for elem in sections]
for i in range(0,len(lst_sec)):
    
    if i == (len(lst_sec)-1):
        print(lst_sec[i].group())
        print("processus : ", title.search(lst_sec[i].group()).group(1))
        df.append(title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        str_between = ch[int(lst_sec[i].span()[1]):]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtitles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
        #####End liste des inputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtitles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"
        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
        ######End liste des tt
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtitles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
        ######End liste des outputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro

    else:
        
        print(lst_sec[i].group())
        # print("processus : ", title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        df.append(title.search(lst_sec[i].group()).group(1))
        str_between = ch[int(lst_sec[i].span()[1]):int(lst_sec[i+1].span()[0])]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtitles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
        #####End liste des inputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtitles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"
        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
        ######End liste des tt
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtitles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
        ######End liste des outputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro
        print('-'*70)

11.6.1 control risks: Inputs

processus :  control risks
	11.6.1.1 Project Management Plan
	11.6.1.2 risk register
	11.6.1.3 Work Performance data
	11.6.1.4 Work Performance reports
11.6.2 control risks: tools and techniques

	11.6.2.1 risk reassessment
	11.6.2.2 risk Audits
	11.6.2.3 Variance and trend Analysis
	11.6.2.4 technical Performance Measurement
	11.6.2.5 reserve Analysis
	11.6.2.6 Meetings
11.6.3 control risks: outputs

	11.6.3.1 Work Performance Information
	11.6.3.2 change requests
	11.6.3.3 Project Management Plan updates
	11.6.3.4 Project documents updates
	11.6.3.5 organizational Process Assets updates


In [71]:
for i in inputs:
    a=i[0]
    exec('%s = %s' %(a, i[:]))
print(outputss)

[['p1_outputs', 'Work Performance Information', 'change requests', 'Project Management Plan updates', 'Project documents updates', 'organizational Process Assets updates']]


In [72]:
for i in outputss:
    a=i[0]
    exec('%s = %s' %(a, i[:]))


In [73]:
for i in tt:
    a=i[0]
    exec('%s = %s' %(a, i[:]))


In [74]:
input_df=pd.DataFrame()
input_df["individual"]=inputs[0]
input_df = input_df.drop(input_df.index[0])
p_m_p=['The project management plan provides guidance for risk monitoring and controlling','The risk register has key inputs that include identified risks and risk owners','Work performance data related to various performance results possibly impacted by risks includes, but is not limited to Deliverable status Schedule progress Costs incurred','Work performance reports take information from performance measurements and analyze it to provide project work performance information']
input_df['annotation']=p_m_p
input_df=input_df.reset_index(drop=True)
p_m_p=['Control Risk Inputs','Control Risk Inputs','Control Risk Inputs','Control Risk Inputs']
input_df['concept']=p_m_p
input_df

,individual,annotation,concept
0,Project Management Plan,The project management plan provides guidance ...,Control Risk Inputs
1,risk register,The risk register has key inputs that include ...,Control Risk Inputs
2,Work Performance data,Work performance data related to various perfo...,Control Risk Inputs
3,Work Performance reports,Work performance reports take information from...,Control Risk Inputs


In [75]:
output_df=pd.DataFrame()
output_df["individual"]=outputss[0]
output_df = output_df.drop(output_df.index[0])
output_df=output_df.reset_index(drop=True)
p_m_p=['Work performance information provides a mechanism to communicate and support project decision making','Change requests are prepared and submitted to the Perform Integrated Change Control process','the corresponding component documents of the project management plan are revised and reissued to reflect the approved changes','Project documents that may be updated as a result of the Control Risk process include  Outcomes of risk reassessments, risk audits, and periodic risk reviews and Actual outcomes of the project’s risks and of the risk responses','The organizational process assets that may be updated include Templates for the risk management plan Risk breakdown structure Lessons learned']
output_df['annotation']=p_m_p
p_m_p=['Control Risk Outputs','Control Risk Outputs','Control Risk Outputs','Control Risk Outputs','Control Risk Outputs']
output_df['concept']=p_m_p
output_df

,individual,annotation,concept
0,Work Performance Information,Work performance information provides a mechan...,Control Risk Outputs
1,change requests,Change requests are prepared and submitted to ...,Control Risk Outputs
2,Project Management Plan updates,the corresponding component documents of the p...,Control Risk Outputs
3,Project documents updates,Project documents that may be updated as a res...,Control Risk Outputs
4,organizational Process Assets updates,The organizational process assets that may be ...,Control Risk Outputs


In [76]:
tools_and_techniques_df=pd.DataFrame()
tools_and_techniques_df["individual"]=tt[0]
tools_and_techniques_df = tools_and_techniques_df.drop(tools_and_techniques_df.index[0])
tools_and_techniques_df=tools_and_techniques_df.reset_index(drop=True)
p_m_p=['Project risk reassessments should be regularly scheduled','Risk audits examine and document the effectiveness of risk responses in dealing with identified risks and their root causes, as well as the effectiveness of the risk management process','Earned value analysis and other methods of project variance and trend analysis may be used for monitoring overall project performance','Technical performance measurement compares technical accomplishments during project execution to the schedule of technical achievement','Reserve analysis compares the amount of the contingency reserves remaining to the amount of risk remaining at any time in the project in order to determine if the remaining reserve is adequate',' Frequent discussions about risk make it more likely that people will identify risks and opportunities']
tools_and_techniques_df['annotation']=p_m_p
p_m_p=['Control Risk tools_and_techniques','Control Risk tools_and_techniques','Control Risk tools_and_techniques','Control Risk tools_and_techniques','Control Risk tools_and_techniques','Control Risk tools_and_techniques']
tools_and_techniques_df['concept']=p_m_p
tools_and_techniques_df

,individual,annotation,concept
0,risk reassessment,Project risk reassessments should be regularly...,Control Risk tools_and_techniques
1,risk Audits,Risk audits examine and document the effective...,Control Risk tools_and_techniques
2,Variance and trend Analysis,Earned value analysis and other methods of pro...,Control Risk tools_and_techniques
3,technical Performance Measurement,Technical performance measurement compares tec...,Control Risk tools_and_techniques
4,reserve Analysis,Reserve analysis compares the amount of the co...,Control Risk tools_and_techniques
5,Meetings,Frequent discussions about risk make it more ...,Control Risk tools_and_techniques


In [77]:
concept_df=pd.DataFrame()
concept = ['Control Risk','Control Risk Inputs','Control Risk Outputs','Control Risk tools_and_techniques']
concept_df["concept"]=concept
p_m_p=['Control Risks is the process of implementing risk response plans, tracking identified risks, monitoring residual risks, identifying new risks, and evaluating risk process effectiveness throughout the project','','','']
concept_df['annotation']=p_m_p
concept_df

,concept,annotation
0,Control Risk,Control Risks is the process of implementing r...
1,Control Risk Inputs,
2,Control Risk Outputs,
3,Control Risk tools_and_techniques,


In [78]:
import PyPDF2

In [79]:
def Split(text):
    x=re.findall('.[^A-Z]*', text)
    x = ' '.join(x) 
    return x
def syns(s):
    l=[]
    for w in nlp(s):
        syn=wordnet.synsets(w.text)
        a=w.text
        if len(syn)>0 and w.pos_ in ['NOUN','VERB']:
            a=syn[0].name().split('.')[0]
        l.append(a)
    return l
def stopwords(s):
    doc=nlp(s)
    tokens = [token.text for token in doc]
    filtered = [token.text for token in doc if token.is_stop == False]
    return filtered
def postag(s):
    doc=nlp(s)
    pos = [[token.text,token.pos_] for token in doc]
    return pos
lemmatizer=WordNetLemmatizer()
def lemm(s):
    l=[]
    for word,tag in pos_tag(word_tokenize(s)):
        wntag=tag[0].lower()
        wntag=wntag if wntag in ['a','r','n','v'] else None
        lemma=lemmatizer.lemmatize(word,wntag) if wntag else word
        l.append(lemma)
    return l
def entity(text):
    doc=nlp(text)
    return [e for e in doc.ents]

def similarity(text1,text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)
def extractVerbNoun(s):
    doc=nlp(s)
    sent=[]
    for token in doc:
        
        if(token.pos_=='AUX') or (token.pos_=='VERB') :
            phrase=' '
            for sub_tok in token.lefts:
                if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    phrase+=sub_tok.text
                    phrase+=' '+token.text
                    for sub_tok in token.rights:
                        if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                            phrase+=' '+sub_tok.text
                            sent.append(phrase)
                            
    return sent
def punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s


In [80]:
nlp = spacy.load("en_core_web_md")

In [81]:
input_df['annotation']=input_df['annotation'].apply(lambda s : "".join(punct(s)))
input_df['annotation']=input_df['annotation'].apply(lambda s : "".join(Split(s)))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s : " ".join(lemm(s)))
input_df['pattern']=input_df["annotation"].apply(lambda s: extractVerbNoun(s))
input_df.style.set_properties(subset=['annotation'])


,individual,annotation,concept,pattern
0,Project Management Plan,project management plan provide guidance risk monitoring control,Control Risk Inputs,[' plan provide control']
1,risk register,risk register have input include identified risk risk owner,Control Risk Inputs,[' register have input']
2,Work Performance data,Work performance data relate performance result impact risk include be Deliverable status Schedule progress Costs incur,Control Risk Inputs,[' data relate result']
3,Work Performance reports,Work performance report take information performance measurement analyze it provide project work performance information,Control Risk Inputs,"[' report take measurement', ' it provide information']"


In [82]:
input_df['pattern'][3]=[' it provide information']
input_df.style.set_properties(subset=['annotation'])

,individual,annotation,concept,pattern
0,Project Management Plan,project management plan provide guidance risk monitoring control,Control Risk Inputs,[' plan provide control']
1,risk register,risk register have input include identified risk risk owner,Control Risk Inputs,[' register have input']
2,Work Performance data,Work performance data relate performance result impact risk include be Deliverable status Schedule progress Costs incur,Control Risk Inputs,[' data relate result']
3,Work Performance reports,Work performance report take information performance measurement analyze it provide project work performance information,Control Risk Inputs,[' it provide information']


In [83]:
output_df['annotation']=output_df['annotation'].apply(lambda s : "".join(punct(s)))
output_df['annotation']=output_df['annotation'].apply(lambda s : "".join(Split(s)))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s : " ".join(lemm(s)))
output_df['pattern']=output_df["annotation"].apply(lambda s: extractVerbNoun(s))
output_df.style.set_properties(subset=['annotation'])

,individual,annotation,concept,pattern
0,Work Performance Information,Work performance information provide mechanism communicate support project decision making,Control Risk Outputs,[' information provide mechanism']
1,change requests,Change request be prepared submit Perform Integrated Change Control process,Control Risk Outputs,[]
2,Project Management Plan updates,correspond component document project management plan be revise reissued reflect approve change,Control Risk Outputs,[]
3,Project documents updates,Project document may be update result Control Risk process include Outcomes risk reassessment risk audit risk review outcome project risk risk response,Control Risk Outputs,"[' process include Outcomes', ' process include Outcomes response']"
4,organizational Process Assets updates,process asset may be update include Templates risk management plan Risk breakdown structure Lessons learn,Control Risk Outputs,[]


In [84]:
output_df['pattern'][1]=[' requests prepare change']
output_df['pattern'][2]=[' updates revise change']
output_df['pattern'][3]=[' documments include response']
output_df['pattern'][4]=[' assets include structure']
output_df.style.set_properties(subset=['annotation'])

,individual,annotation,concept,pattern
0,Work Performance Information,Work performance information provide mechanism communicate support project decision making,Control Risk Outputs,[' information provide mechanism']
1,change requests,Change request be prepared submit Perform Integrated Change Control process,Control Risk Outputs,[' requests prepare change']
2,Project Management Plan updates,correspond component document project management plan be revise reissued reflect approve change,Control Risk Outputs,[' updates revise change']
3,Project documents updates,Project document may be update result Control Risk process include Outcomes risk reassessment risk audit risk review outcome project risk risk response,Control Risk Outputs,[' documments include response']
4,organizational Process Assets updates,process asset may be update include Templates risk management plan Risk breakdown structure Lessons learn,Control Risk Outputs,[' assets include structure']


In [85]:
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : "".join(punct(s)))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : "".join(Split(s)))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : " ".join(lemm(s)))
tools_and_techniques_df['pattern']=tools_and_techniques_df["annotation"].apply(lambda s: extractVerbNoun(s))
tools_and_techniques_df.style.set_properties(subset=['annotation'])


,individual,annotation,concept,pattern
0,risk reassessment,Project risk reassessment should be schedule,Control Risk tools_and_techniques,[' reassessment be schedule']
1,risk Audits,Risk audit examine document effectiveness risk response deal identified risk their root cause effectiveness risk management process,Control Risk tools_and_techniques,[' deal identified risk']
2,Variance and trend Analysis,Earned value analysis method project variance trend analysis may be use monitoring project performance,Control Risk tools_and_techniques,[]
3,technical Performance Measurement,performance measurement compare accomplishment project execution schedule achievement,Control Risk tools_and_techniques,[' measurement compare achievement']
4,reserve Analysis,Reserve analysis compare amount contingency reserve remain amount risk remain time project order determine remain reserve be,Control Risk tools_and_techniques,"[' analysis compare reserve', ' risk remain order']"
5,Meetings,discussion risk make it more people will identify risk opportunity,Control Risk tools_and_techniques,[' people identify opportunity']


In [86]:
tools_and_techniques_df['pattern'][2]=[' variance monitore performance']
tools_and_techniques_df['pattern'][4]=[' analysis compare reserve']
tools_and_techniques_df.style.set_properties(subset=['annotation'])

,individual,annotation,concept,pattern
0,risk reassessment,Project risk reassessment should be schedule,Control Risk tools_and_techniques,[' reassessment be schedule']
1,risk Audits,Risk audit examine document effectiveness risk response deal identified risk their root cause effectiveness risk management process,Control Risk tools_and_techniques,[' deal identified risk']
2,Variance and trend Analysis,Earned value analysis method project variance trend analysis may be use monitoring project performance,Control Risk tools_and_techniques,[' variance monitore performance']
3,technical Performance Measurement,performance measurement compare accomplishment project execution schedule achievement,Control Risk tools_and_techniques,[' measurement compare achievement']
4,reserve Analysis,Reserve analysis compare amount contingency reserve remain amount risk remain time project order determine remain reserve be,Control Risk tools_and_techniques,[' analysis compare reserve']
5,Meetings,discussion risk make it more people will identify risk opportunity,Control Risk tools_and_techniques,[' people identify opportunity']


In [87]:
concept_df['annotation']=concept_df['annotation'].apply(lambda s : "".join(punct(s)))
concept_df['annotation']=concept_df['annotation'].apply(lambda s : "".join(Split(s)))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s : " ".join(lemm(s)))
concept_df['pattern']=concept_df["annotation"].apply(lambda s: extractVerbNoun(s))
concept_df.style.set_properties(subset=['annotation'])


,concept,annotation,pattern
0,Control Risk,Control Risks be process implement risk response plan track identified risk monitor risk identify risk evaluate risk process effectiveness project,"[' Risks implement track', ' Risks implement trackprocess implement track', ' risk evaluate project']"
1,Control Risk Inputs,,[]
2,Control Risk Outputs,,[]
3,Control Risk tools_and_techniques,,[]


In [88]:
concept_df['pattern'][0]=['risk evaluate project']
concept_df.style.set_properties(subset=['annotation'])

,concept,annotation,pattern
0,Control Risk,Control Risks be process implement risk response plan track identified risk monitor risk identify risk evaluate risk process effectiveness project,['risk evaluate project']
1,Control Risk Inputs,,[]
2,Control Risk Outputs,,[]
3,Control Risk tools_and_techniques,,[]


In [89]:
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "Processus", "pattern": [{"lower": {"IN": ["control", "risk"]}},{"lower": {"IN": ["risk", "control"]}}]},
            {"label": "Input", "pattern": [{"lower": {"IN": ["project", "management","plan"]}}, {"lower": {"IN": ["project", "management","plan"]}},{"lower": {"IN": ["project", "management","plan"]}}]},
            {"label": "Output", "pattern": [{"lower": {"IN": ["risk", "reassessment"]}},{"lower": {"IN": ["risk", "reassessment"]}}]},
            {"label": "T_T", "pattern": [{"lower": {"IN": ["change", "requests"]}},{"lower": {"IN": ["change", "requests"]}}]}]
             
ruler.add_patterns(patterns)



In [90]:
doc = nlp("control risk   project management plan   risk reassessment  change requests")

print([(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,style="ent")

[('control risk', 'Processus'), ('project management plan', 'Input'), ('risk reassessment', 'Output'), ('change requests', 'T_T')]


In [94]:
L1=[]
L2=[]
L3=[]

for i in range(len(input_df)):
  L1.append(input_df["individual"][i])
  L2.append(input_df["pattern"][i][0].split()[1])
  L3.append(input_df["pattern"][i][0].split()[2])

input_df["subject"]=L1
input_df["property"]=L2
input_df["object"]=L3

input_df

,individual,annotation,concept,pattern,subject,property,object
0,Project Management Plan,project management plan provide guidance risk ...,Control Risk Inputs,[ plan provide control],Project Management Plan,provide,control
1,risk register,risk register have input include identified ri...,Control Risk Inputs,[ register have input],risk register,have,input
2,Work Performance data,Work performance data relate performance resul...,Control Risk Inputs,[ data relate result],Work Performance data,relate,result
3,Work Performance reports,Work performance report take information perfo...,Control Risk Inputs,[ it provide information],Work Performance reports,provide,information


In [95]:
L1=[]
L2=[]
L3=[]

for i in range(len(output_df)):
  L1.append(output_df["individual"][i])
  L2.append(output_df["pattern"][i][0].split()[1])
  L3.append(output_df["pattern"][i][0].split()[2])

output_df["subject"]=L1
output_df["property"]=L2
output_df["object"]=L3

output_df

,individual,annotation,concept,pattern,subject,property,object
0,Work Performance Information,Work performance information provide mechanism...,Control Risk Outputs,[ information provide mechanism],Work Performance Information,provide,mechanism
1,change requests,Change request be prepared submit Perform Inte...,Control Risk Outputs,[ requests prepare change],change requests,prepare,change
2,Project Management Plan updates,correspond component document project manageme...,Control Risk Outputs,[ updates revise change],Project Management Plan updates,revise,change
3,Project documents updates,Project document may be update result Control ...,Control Risk Outputs,[ documments include response],Project documents updates,include,response
4,organizational Process Assets updates,process asset may be update include Templates ...,Control Risk Outputs,[ assets include structure],organizational Process Assets updates,include,structure


In [96]:
L1=[]
L2=[]
L3=[]

for i in range(len(tools_and_techniques_df)):
  L1.append(tools_and_techniques_df["individual"][i])
  L2.append(tools_and_techniques_df["pattern"][i][0].split()[1])
  L3.append(tools_and_techniques_df["pattern"][i][0].split()[2])

tools_and_techniques_df["subject"]=L1
tools_and_techniques_df["property"]=L2
tools_and_techniques_df["object"]=L3

tools_and_techniques_df

,individual,annotation,concept,pattern,subject,property,object
0,risk reassessment,Project risk reassessment should be schedule,Control Risk tools_and_techniques,[ reassessment be schedule],risk reassessment,be,schedule
1,risk Audits,Risk audit examine document effectiveness risk...,Control Risk tools_and_techniques,[ deal identified risk],risk Audits,identified,risk
2,Variance and trend Analysis,Earned value analysis method project variance ...,Control Risk tools_and_techniques,[ variance monitore performance],Variance and trend Analysis,monitore,performance
3,technical Performance Measurement,performance measurement compare accomplishment...,Control Risk tools_and_techniques,[ measurement compare achievement],technical Performance Measurement,compare,achievement
4,reserve Analysis,Reserve analysis compare amount contingency re...,Control Risk tools_and_techniques,[ analysis compare reserve],reserve Analysis,compare,reserve
5,Meetings,discussion risk make it more people will ident...,Control Risk tools_and_techniques,[ people identify opportunity],Meetings,identify,opportunity


In [100]:
concept_df

,concept,annotation,pattern
0,Control Risk,Control Risks be process implement risk respon...,[risk evaluate project]
1,Control Risk Inputs,,[]
2,Control Risk Outputs,,[]
3,Control Risk tools_and_techniques,,[]


In [107]:
L1=['Control Risk','','','']
L2=['evaluate ','','','']
L3=['project','','','']



concept_df["subject"]=L1
concept_df["property"]=L2
concept_df["object"]=L3

concept_df

,concept,annotation,pattern,subject,property,object
0,Control Risk,Control Risks be process implement risk respon...,[risk evaluate project],Control Risk,evaluate,project
1,Control Risk Inputs,,[],,,
2,Control Risk Outputs,,[],,,
3,Control Risk tools_and_techniques,,[],,,


In [108]:
concept_df.to_csv("concept.csv")
tools_and_techniques_df.to_csv("t_t.csv")
output_df.to_csv("output.csv")
input_df.to_csv("input.csv")